In [1]:
import pandas as pd

# load csvs
idh = pd.read_csv("datasets/human-development-index-cleaned.csv")
schooling = pd.read_csv("datasets/expected-years-of-schooling-cleaned.csv")
gni = pd.read_csv("datasets/gross-national-income-per-capita-cleaned.csv")
population = pd.read_csv("datasets/population_total_long-cleaned.csv")
alpha3 = pd.read_csv("datasets/iso_noc-merged.csv")


In [2]:
population

,Country Name,Year,Count,Code
0,Aruba,1960,54211,ABW
1,Afghanistan,1960,8996973,AFG
2,Angola,1960,5454933,AGO
3,Albania,1960,1608800,ALB
4,Andorra,1960,13411,AND
...,...,...,...,...
12590,Kosovo,2017,1830700,XKX
12591,"Yemen, Rep.",2017,27834821,YEM
12592,South Africa,2017,57000451,ZAF
12593,Zambia,2017,16853688,ZMB


In [3]:
data_merged = population.merge(
    idh[["Year", "Code", "Human Development Index (UNDP)"]],
    on=["Year", "Code"],
    how="left",
)
data_merged = data_merged.merge(
    schooling[["Year", "Code", "Expected Years of Schooling (years)"]],
    on=["Year", "Code"],
    how="left",
)
data_merged = data_merged.merge(
    gni[["Year", "Code", "GNI per capita, PPP (constant 2017 international $)"]],
    on=["Year", "Code"],
    how="left",
)


In [4]:
# get all entires with nan values in the Code column
nan_codes = data_merged[data_merged["Code"].isna()]
nan_codes


,Country Name,Year,Count,Code,Human Development Index (UNDP),Expected Years of Schooling (years),"GNI per capita, PPP (constant 2017 international $)"


In [5]:
data_merged

,Country Name,Year,Count,Code,Human Development Index (UNDP),Expected Years of Schooling (years),"GNI per capita, PPP (constant 2017 international $)"
0,Aruba,1960,54211,ABW,NaN,NaN,NaN
1,Afghanistan,1960,8996973,AFG,NaN,NaN,NaN
2,Angola,1960,5454933,AGO,NaN,NaN,NaN
3,Albania,1960,1608800,ALB,NaN,NaN,NaN
4,Andorra,1960,13411,AND,NaN,NaN,NaN
...,...,...,...,...,...,...,...
12590,Kosovo,2017,1830700,XKX,NaN,NaN,NaN
12591,"Yemen, Rep.",2017,27834821,YEM,0.452,9.0,NaN
12592,South Africa,2017,57000451,ZAF,0.699,13.3,12320.426838
12593,Zambia,2017,16853688,ZMB,0.588,12.5,3330.552717


In [6]:
# show entries after 1990
data_merged[data_merged["Year"] > 1990]


,Country Name,Year,Count,Code,Human Development Index (UNDP),Expected Years of Schooling (years),"GNI per capita, PPP (constant 2017 international $)"
6698,Aruba,1991,64622,ABW,NaN,NaN,NaN
6699,Afghanistan,1991,13299017,AFG,NaN,2.9,NaN
6700,Angola,1991,12248901,AGO,NaN,3.6,NaN
6701,Albania,1991,3266790,ALB,0.626,11.8,NaN
6702,Andorra,1991,56671,AND,NaN,10.8,NaN
...,...,...,...,...,...,...,...
12590,Kosovo,2017,1830700,XKX,NaN,NaN,NaN
12591,"Yemen, Rep.",2017,27834821,YEM,0.452,9.0,NaN
12592,South Africa,2017,57000451,ZAF,0.699,13.3,12320.426838
12593,Zambia,2017,16853688,ZMB,0.588,12.5,3330.552717


In [7]:
# save to csv file
data_merged.to_csv("datasets/country-data-merged.csv", index=False)
